In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spectools.lpsd as lpsd
import spectools.dsp as dsp 
from functools import partial
import multiprocessing as mp
ctx = mp.get_context('fork')
pool = ctx.Pool()
import requests
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

In [ ]:
# The following links should lead to two zip files, each containing one txt file
# Contact the author at spectools@pm.me if the links are broken
url_list = []
url_list.append('https://www.dropbox.com/scl/fi/tjohxdn6s1kj4c47qhx38/2024_09_12_09_40_57_raw.txt.zip?rlkey=7tmcup1n70ffquyigpmia2oec&dl=1')
url_list.append('https://www.dropbox.com/scl/fi/qe2obv83gzi842cyvbaqi/2024_09_12_09_40_58_temp_data.txt.zip?rlkey=ll3k3xneojhvw7z3qwguuwzei&dl=1')

file_list = []
file_list.append('2024_09_12_09_40_57_raw.txt.zip')
file_list.append('2024_09_12_09_40_58_temp_data.txt.zip')

for url, file in zip(url_list, file_list):
    if os.path.exists(file):
        print(f"{file} already exists. Skipping download.")
        continue

    response = requests.get(url, stream=True)  # Stream the response
    if response.status_code == 200:
        total_size = int(response.headers.get('content-length', 0))  # Total file size in bytes
        chunk_size = 1024  # Chunk size for progress bar
        progress = tqdm(total=total_size, unit='B', unit_scale=True, desc="Downloading")
        
        # Write the file in chunks to a local file
        with open(file, "wb") as f:
            for chunk in response.iter_content(chunk_size=chunk_size):
                f.write(chunk)
                progress.update(len(chunk))
        
        progress.close()
        print(f"File downloaded successfully as {file}!")
    else:
        raise Exception(f"Failed to download file. Status code: {response.status_code}")

In [ ]:
# Phase readout metadata definitions:
sps = 80e6     # Sampling rate of ADC (and PM digital clock) (Hz)
CIC_R = 2**17  # CIC filter
FIR1_R = 6     # First FIR from 610Hz to 102Hz
FIR2_R = 10    # Second FIR from 102Hz to 10Hz, 104 coefficients
FIR3_R = 3     # Third FIR from 10Hz to 3Hz, 77 coefficients
total_R = CIC_R * FIR1_R * FIR2_R * FIR3_R # Total down-sampling rate
fs_pm = sps/total_R # Final data rate (= PM output rate)

pir_1a = '1_main_a_pir[Hz]'
pir_1b = '1_main_b_pir[Hz]'
pir_1c = '1_main_c_pir[Hz]'
pir_1d = '1_main_d_pir[Hz]'
pir_2a = '2_main_a_pir[Hz]'
pir_2b = '2_main_b_pir[Hz]'
pir_2c = '2_main_c_pir[Hz]'
pir_2d = '2_main_d_pir[Hz]'
pir_1a_pt = '1_pilot_a_pir[Hz]'
pir_1b_pt = '1_pilot_b_pir[Hz]'
pir_1c_pt = '1_pilot_c_pir[Hz]'
pir_1d_pt = '1_pilot_d_pir[Hz]'
pir_2a_pt = '2_pilot_a_pir[Hz]'
pir_2b_pt = '2_pilot_b_pir[Hz]'
pir_2c_pt = '2_pilot_c_pir[Hz]'
pir_2d_pt = '2_pilot_d_pir[Hz]'
pir_1a_corr = '1_main_a_pir[Hz]_corr'
pir_1b_corr = '1_main_b_pir[Hz]_corr'
pir_1c_corr = '1_main_c_pir[Hz]_corr'
pir_1d_corr = '1_main_d_pir[Hz]_corr'
pir_2a_corr = '2_main_a_pir[Hz]_corr'
pir_2b_corr = '2_main_b_pir[Hz]_corr'
pir_2c_corr = '2_main_c_pir[Hz]_corr'
pir_2d_corr = '2_main_d_pir[Hz]_corr'
phase_1a = '1_a_phase[rad]'
phase_1b = '1_b_phase[rad]'
phase_1c = '1_c_phase[rad]'
phase_1d = '1_d_phase[rad]'
phase_2a = '2_a_phase[rad]'
phase_2b = '2_b_phase[rad]'
phase_2c = '2_c_phase[rad]'
phase_2d = '2_d_phase[rad]'
phase_1a_corr = '1_a_phase[rad]_corr'
phase_1b_corr = '1_b_phase[rad]_corr'
phase_1c_corr = '1_c_phase[rad]_corr'
phase_1d_corr = '1_d_phase[rad]_corr'
phase_2a_corr = '2_a_phase[rad]_corr'
phase_2b_corr = '2_b_phase[rad]_corr'
phase_2c_corr = '2_c_phase[rad]_corr'
phase_2d_corr = '2_d_phase[rad]_corr'
phase_1a_fluc = '1_a_phase[rad]_fluc'
phase_1b_fluc = '1_b_phase[rad]_fluc'
phase_1c_fluc = '1_c_phase[rad]_fluc'
phase_1d_fluc = '1_d_phase[rad]_fluc'
phase_2a_fluc = '2_a_phase[rad]_fluc'
phase_2b_fluc = '2_b_phase[rad]_fluc'
phase_2c_fluc = '2_c_phase[rad]_fluc'
phase_2d_fluc = '2_d_phase[rad]_fluc'
phase_1a_fluc_corr = '1_a_phase[rad]_fluc_corr'
phase_1b_fluc_corr = '1_b_phase[rad]_fluc_corr'
phase_1c_fluc_corr = '1_c_phase[rad]_fluc_corr'
phase_1d_fluc_corr = '1_d_phase[rad]_fluc_corr'
phase_2a_fluc_corr = '2_a_phase[rad]_fluc_corr'
phase_2b_fluc_corr = '2_b_phase[rad]_fluc_corr'
phase_2c_fluc_corr = '2_c_phase[rad]_fluc_corr'
phase_2d_fluc_corr = '2_d_phase[rad]_fluc_corr'
phase_1ab = 'phase_1a_minus_1b[rad]_fluc'
phase_1ac = 'phase_1a_minus_1c[rad]_fluc'
phase_1ad = 'phase_1a_minus_1d[rad]_fluc'
phase_1bc = 'phase_1b_minus_1c[rad]_fluc'
phase_1bd = 'phase_1b_minus_1d[rad]_fluc'
phase_1cd = 'phase_1c_minus_1d[rad]_fluc'
phase_2ab = 'phase_2a_minus_2b[rad]_fluc'
phase_2ac = 'phase_2a_minus_2c[rad]_fluc'
phase_2ad = 'phase_2a_minus_2d[rad]_fluc'
phase_2bc = 'phase_2b_minus_2c[rad]_fluc'
phase_2bd = 'phase_2b_minus_2d[rad]_fluc'
phase_2cd = 'phase_2c_minus_2d[rad]_fluc'
phase_1ab_corr = 'phase_1a_minus_1b[rad]_fluc_corr'
phase_1ac_corr = 'phase_1a_minus_1c[rad]_fluc_corr'
phase_1ad_corr = 'phase_1a_minus_1d[rad]_fluc_corr'
phase_1bc_corr = 'phase_1b_minus_1c[rad]_fluc_corr'
phase_1bd_corr = 'phase_1b_minus_1d[rad]_fluc_corr'
phase_1cd_corr = 'phase_1c_minus_1d[rad]_fluc_corr'
phase_2ab_corr = 'phase_2a_minus_2b[rad]_fluc_corr'
phase_2ac_corr = 'phase_2a_minus_2c[rad]_fluc_corr'
phase_2ad_corr = 'phase_2a_minus_2d[rad]_fluc_corr'
phase_2bc_corr = 'phase_2b_minus_2c[rad]_fluc_corr'
phase_2bd_corr = 'phase_2b_minus_2d[rad]_fluc_corr'
phase_2cd_corr = 'phase_2c_minus_2d[rad]_fluc_corr'
dws_1v = '1_dws_v[rad]'
dws_1h = '1_dws_h[rad]'
dws_1e = '1_dws_e[rad]'
dws_2v = '2_dws_v[rad]'
dws_2h = '2_dws_h[rad]'
dws_2e = '2_dws_e[rad]'
dws_Dh = 'D_dws_h[rad]'
dws_Dv = 'D_dws_v[rad]'
dws_De = 'D_dws_e[rad]'
dws_1v_corr = '1_dws_v[rad]_corr'
dws_1h_corr = '1_dws_h[rad]_corr'
dws_1e_corr = '1_dws_e[rad]_corr'
dws_2v_corr = '2_dws_v[rad]_corr'
dws_2h_corr = '2_dws_h[rad]_corr'
dws_2e_corr = '2_dws_e[rad]_corr'
dws_Dh_corr = 'D_dws_h[rad]_corr'
dws_Dv_corr = 'D_dws_v[rad]_corr'
dws_De_corr = 'D_dws_e[rad]_corr'
lps_1 = '1_lps[rad]'
lps_2 = '2_lps[rad]'
lps_D = 'D_lps[rad]'
lps_1_corr = '1_lps[rad]_corr'
lps_2_corr = '2_lps[rad]_corr'
lps_D_corr = 'D_lps[rad]_corr'

pirs_1 = [pir_1a, pir_1b, pir_1c, pir_1d]
pirs_2 = [pir_2a, pir_2b, pir_2c, pir_2d]
pirs_1_pt = [pir_1a_pt, pir_1b_pt, pir_1c_pt, pir_1d_pt]
pirs_2_pt = [pir_2a_pt, pir_2b_pt, pir_2c_pt, pir_2d_pt]
pirs_1_corr = [pir_1a_corr, pir_1b_corr, pir_1c_corr, pir_1d_corr]
pirs_2_corr = [pir_2a_corr, pir_2b_corr, pir_2c_corr, pir_2d_corr]

phases_1 = [phase_1a, phase_1b, phase_1c, phase_1d]
phases_2 = [phase_2a, phase_2b, phase_2c, phase_2d]
phases_1_corr = [phase_1a_corr, phase_1b_corr, phase_1c_corr, phase_1d_corr]
phases_2_corr = [phase_2a_corr, phase_2b_corr, phase_2c_corr, phase_2d_corr]
phases_1_fluc = [phase_1a_fluc, phase_1b_fluc, phase_1c_fluc, phase_1d_fluc]
phases_2_fluc = [phase_2a_fluc, phase_2b_fluc, phase_2c_fluc, phase_2d_fluc]
phases_1_fluc_corr = [phase_1a_fluc_corr, phase_1b_fluc_corr, phase_1c_fluc_corr, phase_1d_fluc_corr]
phases_2_fluc_corr = [phase_2a_fluc_corr, phase_2b_fluc_corr, phase_2c_fluc_corr, phase_2d_fluc_corr]

split_phases_1 = [phase_1ab, phase_1ac, phase_1ad, phase_1bc, phase_1bd, phase_1cd]
split_phases_2 = [phase_2ab, phase_2ac, phase_2ad, phase_2bc, phase_2bd, phase_2cd]
split_phases_1_corr = [phase_1ab_corr, phase_1ac_corr, phase_1ad_corr, phase_1bc_corr, phase_1bd_corr, phase_1cd_corr]
split_phases_2_corr = [phase_2ab_corr, phase_2ac_corr, phase_2ad_corr, phase_2bc_corr, phase_2bd_corr, phase_2cd_corr]

pirs = pirs_1 + pirs_2
pirs_pt = pirs_1_pt + pirs_2_pt
pirs_corr = pirs_1_corr + pirs_2_corr
phases = phases_1 + phases_2
phases_corr = phases_1_corr + phases_2_corr
phases_fluc = phases_1_fluc + phases_2_fluc
phases_fluc_corr = phases_1_fluc_corr + phases_2_fluc_corr
split_phases = split_phases_1 + split_phases_2
split_phases_corr = split_phases_1_corr + split_phases_2_corr
dws_signals_1 = [dws_1v, dws_1h, dws_1e]
dws_signals_2 = [dws_2v, dws_2h, dws_2e]
dws_signals_D = [dws_Dv, dws_Dh, dws_De]
dws_signals_1_corr = [dws_1v_corr, dws_1h_corr, dws_1e_corr]
dws_signals_2_corr = [dws_2v_corr, dws_2h_corr, dws_2e_corr]
dws_signals_D_corr = [dws_Dv_corr, dws_Dh_corr, dws_De_corr]
dws_signals = dws_signals_1 + dws_signals_2 + dws_signals_D
dws_signals_corr = dws_signals_1_corr + dws_signals_2_corr + dws_signals_D_corr
dws_signals_v = [dws_1v, dws_2v]
dws_signals_h = [dws_1h, dws_2h]
dws_signals_e = [dws_1e, dws_2e]
dws_signals_v_corr = [dws_1v_corr, dws_2v_corr]
dws_signals_h_corr = [dws_1h_corr, dws_2h_corr]
dws_signals_e_corr = [dws_1e_corr, dws_2e_corr]
lps_signals = [lps_1, lps_2, lps_D]
lps_signals_corr = [lps_1_corr, lps_2_corr, lps_D_corr]

In [ ]:
# Phase readout pre-processing:
def pms_processor(
        df: pd.DataFrame, pt_frequency: float, pir_detrend_order: int = 0, phase_detrend_order: int = 0,
        skip_adc1: bool = False, skip_adc2: bool = False, pt_correction_fun: callable = None,
        skip_pt: bool = False, skip_split: bool = False, skip_lps: bool = False, skip_dws: bool = False
        ) -> pd.DataFrame:
    pm = df.copy()

    pt_correction_fun = pt_correction_fun or (lambda PIR, PT, PT_frequency: PIR * (2.0 - PT / PT_frequency))    

    if skip_adc1 and skip_adc2:
        return pm
    elif skip_adc1 and not skip_adc2:
        pirs = pirs_2
        pirs_pt = pirs_2_pt
        pirs_corr = pirs_2_corr
        phases = phases_2
        phases_corr = phases_2_corr
        phases_fluc = phases_2_fluc
        phases_fluc_corr = phases_2_fluc_corr
    elif skip_adc2 and not skip_adc1:
        pirs = pirs_1
        pirs_pt = pirs_1_pt
        pirs_corr = pirs_1_corr
        phases = phases_1
        phases_corr = phases_1_corr
        phases_fluc = phases_1_fluc
        phases_fluc_corr = phases_1_fluc_corr
    else:
        pirs = pirs_1 + pirs_2
        pirs_pt = pirs_1_pt + pirs_2_pt
        pirs_corr = pirs_1_corr + pirs_2_corr
        phases = phases_1 + phases_2
        phases_corr = phases_1_corr + phases_2_corr
        phases_fluc = phases_1_fluc + phases_2_fluc
        phases_fluc_corr = phases_1_fluc_corr + phases_2_fluc_corr

    n = len(pm)
    x = np.arange(n)
    for (pir, pt, pir_corr, phase, phase_fluc, phase_corr, phase_fluc_corr)\
        in zip(pirs, pirs_pt, pirs_corr, phases, phases_fluc, phases_corr, phases_fluc_corr):
        PIR = np.array(pm[pir]) # Beatnote PIR value in Hz
        PIRfit = np.polyval(np.polyfit(x, PIR, pir_detrend_order), x)
        pm[phase] = (2*np.pi/fs_pm)*np.cumsum(np.array(PIR))
        pm[phase_fluc] = (2*np.pi/fs_pm)*np.cumsum(np.array(PIR-PIRfit))
        if phase_detrend_order >= 1:
            pm[phase_fluc] -= np.polyval(np.polyfit(x, pm[phase_fluc], phase_detrend_order), x)

        if not skip_pt:
            PT_mean = pm[pt].mean() # PT PIR mean value in Hz
            
            if PT_mean < 0.0 or abs(PT_mean) > sps/2.0: # PT signal is aliased
                if PT_mean < 0.0:
                    PT = np.array(sps + pm[pt])
                else:
                    PT = np.array(sps - pm[pt])
            else: # PT signal is NOT aliased
                PT = np.array(pm[pt])

            PIR_corr = pt_correction_fun(PIR, PT, pt_frequency) # PT-corrected PIR value
            PIR_corr_fit = np.polyval(np.polyfit(x, PIR_corr, pir_detrend_order), x)
            pm[pir_corr] = PIR_corr
            pm[phase_corr] = (2*np.pi/fs_pm)*np.cumsum(np.array(PIR_corr))
            pm[phase_fluc_corr] = (2*np.pi/fs_pm)*np.cumsum(np.array(PIR_corr-PIR_corr_fit))
            if phase_detrend_order >= 1:
                pm[phase_fluc_corr] -= np.polyval(np.polyfit(x, pm[phase_fluc_corr], phase_detrend_order), x)

    if not skip_split: # Non-PT-corrected split measurements
        if not skip_adc1:
            pm[phase_1ab] = pm[phase_1a_fluc] - pm[phase_1b_fluc]
            pm[phase_1ac] = pm[phase_1a_fluc] - pm[phase_1c_fluc]
            pm[phase_1ad] = pm[phase_1a_fluc] - pm[phase_1d_fluc]
            pm[phase_1bc] = pm[phase_1b_fluc] - pm[phase_1c_fluc]
            pm[phase_1bd] = pm[phase_1b_fluc] - pm[phase_1d_fluc]
            pm[phase_1cd] = pm[phase_1c_fluc] - pm[phase_1d_fluc]
        if not skip_adc2:
            pm[phase_2ab] = pm[phase_2a_fluc] - pm[phase_2b_fluc]
            pm[phase_2ac] = pm[phase_2a_fluc] - pm[phase_2c_fluc]
            pm[phase_2ad] = pm[phase_2a_fluc] - pm[phase_2d_fluc]
            pm[phase_2bc] = pm[phase_2b_fluc] - pm[phase_2c_fluc]
            pm[phase_2bd] = pm[phase_2b_fluc] - pm[phase_2d_fluc]
            pm[phase_2cd] = pm[phase_2c_fluc] - pm[phase_2d_fluc]

        if not skip_pt: # PT-corrected split measurements
            if not skip_adc1:
                pm[phase_1ab_corr] = pm[phase_1a_fluc_corr] - pm[phase_1b_fluc_corr]
                pm[phase_1ac_corr] = pm[phase_1a_fluc_corr] - pm[phase_1c_fluc_corr]
                pm[phase_1ad_corr] = pm[phase_1a_fluc_corr] - pm[phase_1d_fluc_corr]
                pm[phase_1bc_corr] = pm[phase_1b_fluc_corr] - pm[phase_1c_fluc_corr]
                pm[phase_1bd_corr] = pm[phase_1b_fluc_corr] - pm[phase_1d_fluc_corr]
                pm[phase_1cd_corr] = pm[phase_1c_fluc_corr] - pm[phase_1d_fluc_corr]
            if not skip_adc2:
                pm[phase_2ab_corr] = pm[phase_2a_fluc_corr] - pm[phase_2b_fluc_corr]
                pm[phase_2ac_corr] = pm[phase_2a_fluc_corr] - pm[phase_2c_fluc_corr]
                pm[phase_2ad_corr] = pm[phase_2a_fluc_corr] - pm[phase_2d_fluc_corr]
                pm[phase_2bc_corr] = pm[phase_2b_fluc_corr] - pm[phase_2c_fluc_corr]
                pm[phase_2bd_corr] = pm[phase_2b_fluc_corr] - pm[phase_2d_fluc_corr]
                pm[phase_2cd_corr] = pm[phase_2c_fluc_corr] - pm[phase_2d_fluc_corr]

    if not skip_dws: # Non-PT-corrected DWS measurements
        if not skip_adc1:
            pm[dws_1h] = 0.5*(pm[phase_1a_fluc] - pm[phase_1b_fluc] + pm[phase_1c_fluc] - pm[phase_1d_fluc])
            pm[dws_1v] = 0.5*(pm[phase_1a_fluc] + pm[phase_1b_fluc] - pm[phase_1c_fluc] - pm[phase_1d_fluc])
            pm[dws_1e] = 0.5*(pm[phase_1a_fluc] - pm[phase_1b_fluc] - pm[phase_1c_fluc] + pm[phase_1d_fluc])
        if not skip_adc2:
            pm[dws_2h] = 0.5*(pm[phase_2a_fluc] - pm[phase_2b_fluc] + pm[phase_2c_fluc] - pm[phase_2d_fluc])
            pm[dws_2v] = 0.5*(pm[phase_2a_fluc] + pm[phase_2b_fluc] - pm[phase_2c_fluc] - pm[phase_2d_fluc])
            pm[dws_2e] = 0.5*(pm[phase_2a_fluc] - pm[phase_2b_fluc] - pm[phase_2c_fluc] + pm[phase_2d_fluc])
        if not skip_adc1 and not skip_adc2:
            pm[dws_Dh] = pm[dws_1h] - pm[dws_2h]
            pm[dws_Dv] = pm[dws_1v] - pm[dws_2v]
            pm[dws_De] = pm[dws_1e] - pm[dws_2e]

        if not skip_pt: # PT-corrected DWS measurements
            if not skip_adc1:
                pm[dws_1h_corr] = 0.5*(pm[phase_1a_fluc_corr] - pm[phase_1b_fluc_corr] + pm[phase_1c_fluc_corr] - pm[phase_1d_fluc_corr])
                pm[dws_1v_corr] = 0.5*(pm[phase_1a_fluc_corr] + pm[phase_1b_fluc_corr] - pm[phase_1c_fluc_corr] - pm[phase_1d_fluc_corr])
                pm[dws_1e_corr] = 0.5*(pm[phase_1a_fluc_corr] - pm[phase_1b_fluc_corr] - pm[phase_1c_fluc_corr] + pm[phase_1d_fluc_corr])
            if not skip_adc2:
                pm[dws_2h_corr] = 0.5*(pm[phase_2a_fluc_corr] - pm[phase_2b_fluc_corr] + pm[phase_2c_fluc_corr] - pm[phase_2d_fluc_corr])
                pm[dws_2v_corr] = 0.5*(pm[phase_2a_fluc_corr] + pm[phase_2b_fluc_corr] - pm[phase_2c_fluc_corr] - pm[phase_2d_fluc_corr])
                pm[dws_2e_corr] = 0.5*(pm[phase_2a_fluc_corr] - pm[phase_2b_fluc_corr] - pm[phase_2c_fluc_corr] + pm[phase_2d_fluc_corr])
            if not skip_adc1 and not skip_adc2:
                pm[dws_Dh_corr] = pm[dws_1h_corr] - pm[dws_2h_corr]
                pm[dws_Dv_corr] = pm[dws_1v_corr] - pm[dws_2v_corr]
                pm[dws_De_corr] = pm[dws_1e_corr] - pm[dws_2e_corr]

    if not skip_lps: # Non-PT-corrected LPS measurements
        if not skip_adc1:
            pm[lps_1] = 0.25*(pm[phase_1a_fluc] + pm[phase_1b_fluc] + pm[phase_1c_fluc] + pm[phase_1d_fluc])
        if not skip_adc2:
            pm[lps_2] = 0.25*(pm[phase_2a_fluc] + pm[phase_2b_fluc] + pm[phase_2c_fluc] + pm[phase_2d_fluc])
        if not skip_adc1 and not skip_adc2:
            pm[lps_D] = pm[lps_1] - pm[lps_2]

        if not skip_pt: # PT-corrected LPS measurements
            if not skip_adc1:
                pm[lps_1_corr] = 0.25*(pm[phase_1a_fluc_corr] + pm[phase_1b_fluc_corr] + pm[phase_1c_fluc_corr] + pm[phase_1d_fluc_corr])
            if not skip_adc2:
                pm[lps_2_corr] = 0.25*(pm[phase_2a_fluc_corr] + pm[phase_2b_fluc_corr] + pm[phase_2c_fluc_corr] + pm[phase_2d_fluc_corr])
            if not skip_adc1 and not skip_adc2:
                pm[lps_D_corr] = pm[lps_1_corr] - pm[lps_2_corr]

    return pm

In [ ]:
# Temperature readout pre-processing:
def temp_processor_fun(df: pd.DataFrame) -> pd.DataFrame:
    temp = df.copy()
    temp = temp.rename(columns={"dev1ch0": "T_GHz_1", 
              "dev1ch1": "T_div_1", 
              "dev1ch2": "T_PTamp_1", 
              "dev1ch3": "T_PTsplitter_1", 
              "dev1ch4": "T_frame_top_1", 
              "dev1ch5": "T_baseplate", 
              "dev1ch6": "Unnamed: 1", 
              "dev1ch7": "T_div_2", 
              "dev2ch0": "T_chamber_L", 
              "dev2ch1": "T_frame_1", 
              "dev2ch2": "T_PTamp_2", 
              "dev2ch3": "T_USO_1", 
              "dev2ch4": "T_PTsplitter_2", 
              "dev2ch5": "T_80MHz_1", 
              "dev2ch6": "Unnamed: 2", 
              "dev2ch7": "T_chamber_R"})
    temp.dropna(axis=1, how='all', inplace=True)
    temp.drop(["Unnamed: 1", "Unnamed: 2"], axis=1, inplace=True)
    temp['T_average_pt1'] = temp[['T_GHz_1', 'T_div_1', 'T_PTamp_1', 'T_PTsplitter_1', 'T_USO_1', 'T_80MHz_1']].sum(axis=1)/4    
    return temp

# Temperature readout sampling frequency:
fs_temp = 3.108

In [ ]:
# Phase readout pre-processor:
pms_processor_fun = partial(pms_processor, 
                            pt_frequency=78e6, 
                            pir_detrend_order=0, 
                            phase_detrend_order=1)

# Sampling frequency in the common time grid:
fs = 2.0

df = dsp.multi_file_timeseries_resampler(
    ['./2024_09_12_09_40_58_temp_data.txt.zip', 
     './2024_09_12_09_40_57_raw.txt.zip'], 
    [fs_temp, fs_pm],  # Sampling frequencies of the input data streams
    fs,                # Sampling frequency in the common time grid
    start_time = 60.0, # Discard the first `start_time` seconds from all data streams
    timeshifts = [0.0, 0.0], # Timeshift each data stream by these number of seconds
    preprocessors = [temp_processor_fun, pms_processor_fun]) # Apply these data pre-processors before resampling

In [ ]:
ifo = 'pt_phase'
T = 'pt_temp'
T_sensor = 'T_average_pt1'
df[ifo] = df[phase_1cd_corr] - np.polyval(np.polyfit(df['common_time'], df[phase_1cd_corr], 1), df['common_time'])
df[T] = df[T_sensor] - (np.max(df[T_sensor]) + np.min(df[T_sensor])) / 2

fig, ax = plt.subplots()
ax.plot(df.common_time/3600, df[T], label=T)
ax.plot(df.common_time/3600, df[ifo]*1.0e3, label=ifo + r" $\times 10^3$")
ax.set_xlabel("Time (hours)")
ax.set_ylabel("Signal")
ax.legend()
plt.show()

In [ ]:
ltf_obj = lpsd.ltf([df[T], df[ifo]], fs=fs, pool=pool)

In [ ]:
fig, (ax2, ax1) = ltf_obj.plot(which='bode', errors=True, sigma=3)
ax1.set_ylim(-640, 0)
plt.show()

In [ ]:
fig, ax = ltf_obj.plot(which='coh', errors=True, sigma=3)
plt.show()